## Car Damage Detection

This notebook implements YOLOv11 detection to detect types of damage on cars using bounding boxes. The dataset comes from https://cardd-ustc.github.io/

### Dependency Imports

In [ ]:
from ultralytics import YOLO
import pandas as pd
import json
import os
import shutil

### Data Preprocessing

In [ ]:
print("Formatting dataset for YOLO detection...")

# Create directory structure for detection
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(f'../../dataset/CarDD_YOLO_Detection/{split}/images', exist_ok=True)
    os.makedirs(f'../../dataset/CarDD_YOLO_Detection/{split}/labels', exist_ok=True)

    # Load COCO annotations
    with open(f'../../dataset/CarDD_COCO/annotations/instances_{split}2017.json', 'r') as f:
        split_data = json.load(f)

    images = split_data['images']
    annotations = split_data['annotations']
    
    # Group annotations by image_id (one image can have multiple annotations)
    annotations_by_image = {}
    for annotation in annotations:
        image_id = annotation['image_id']
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = []
        annotations_by_image[image_id].append(annotation)
    
    print(f"\nProcessing {split} split: {len(images)} images, {len(annotations)} annotations")
    
    for image in images:
        image_name = image['file_name']
        image_id = image['id']
        width = image['width']
        height = image['height']
        
        # Copy image to YOLO directory
        src_path = f'../../dataset/CarDD_COCO/{split}2017/{image_name}'
        dst_path = f'../../dataset/CarDD_YOLO_Detection/{split}/images/{image_name}'
        shutil.copy(src_path, dst_path)
        
        # Create corresponding label file
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_path = f'../../dataset/CarDD_YOLO_Detection/{split}/labels/{label_name}'
        
        # Get all annotations for this image
        image_annotations = annotations_by_image.get(image_id, [])
        
        with open(label_path, 'w') as label_file:
            for annotation in image_annotations:
                # YOLO uses 0-indexed classes, COCO uses 1-indexed
                class_id = annotation['category_id'] - 1
                
                # Get bounding box from COCO format [x, y, width, height] (top-left corner)
                bbox = annotation['bbox']
                x_min, y_min, bbox_width, bbox_height = bbox
                
                # Convert to YOLO format: [center_x, center_y, width, height] (normalized)
                center_x = (x_min + bbox_width / 2) / width
                center_y = (y_min + bbox_height / 2) / height
                norm_width = bbox_width / width
                norm_height = bbox_height / height
                
                # Write to label file: class_id center_x center_y width height
                line = f"{class_id} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}"
                label_file.write(line + '\n')

print("\nDataset conversion complete!")
print("\nCategory mapping (YOLO class_id: name):")
categories = split_data['categories']
for cat in categories:
    print(f"  {cat['id'] - 1}: {cat['name']}")

### Create YOLO Dataset Configuration

In [ ]:
# Create YOLO dataset configuration file for detection
import yaml

dataset_config = {
    'path': '../../dataset/CarDD_YOLO_Detection',  # Dataset root directory
    'train': 'train/images',  # Training images
    'val': 'val/images',      # Validation images
    'test': 'test/images',    # Test images
    
    # Classes
    'names': {
        0: 'dent',
        1: 'scratch',
        2: 'crack',
        3: 'glass shatter',
        4: 'lamp broken',
        5: 'tire flat'
    }
}

config_path = '../../dataset/CarDD_YOLO_Detection/data.yaml'
with open(config_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print(f"YOLO dataset configuration saved to: {config_path}")
print("\nDataset is ready for YOLO detection training!")

### Train YOLOv11 Detection Model

In [ ]:
# Load a pretrained YOLOv11 detection model
model = YOLO('yolo11n.pt')  # nano model for faster training, use yolo11s.pt, yolo11m.pt for better accuracy

# Train the model
results = model.train(
    data='../../dataset/CarDD_YOLO_Detection/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='car_damage_det',
    project='runs/detect',
    patience=10,  # Early stopping patience
    save=True,
    plots=True
)

### Evaluate Model Performance

In [ ]:
# Load the best trained model
best_model = YOLO('runs/detect/car_damage_det/weights/best.pt')

# Validate the model on the test set
metrics = best_model.val(data='../../dataset/CarDD_YOLO_Detection/data.yaml', split='test')

print("\nTest Set Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

### Run Inference on Test Images

In [ ]:
# Run inference on test images
test_image_path = '../../dataset/CarDD_YOLO_Detection/test/images'

# Predict on a batch of test images
results = best_model.predict(
    source=test_image_path,
    save=True,  # Save annotated images
    conf=0.25,  # Confidence threshold
    project='runs/detect',
    name='predictions',
    exist_ok=True
)

print(f"\nProcessed {len(results)} images")
print(f"Results saved to: runs/detect/predictions")

# Display results for first image
from IPython.display import Image, display
import glob

pred_images = glob.glob('runs/detect/predictions/*.jpg')
if pred_images:
    print("\nSample prediction:")
    display(Image(filename=pred_images[0]))